In [20]:

import numpy as np
from itertools import product
from tqdm.notebook import tqdm

"""
S is the set of all possible states
R is the set of all possible rewards
p(s', r | s, a) is the dynamics
list of actions A(s) for all s in S
theta is how well to aproximate the true value funtion.
"""
def policy_iteration(S:[int], R:[int],p:callable, A:callable, gamma:float, theta:float) -> tuple:
    # initalize
    V = dict()
    pi = dict()
    for s in S:
        V[s] = 0
        pi[s] = 0
    
    policy_stable = False
    while not policy_stable:
        # evaluate policy
        while True:
            delta = 0
            for s in tqdm(S):
                v = V[s]
                V[s] = sum([p(s_,r,s,pi[s]) * (r + gamma*V[s]) for s_, r in product(S, R)])
                delta = max(delta, np.abs(v - V[s]))
            print(delta)
            if delta < theta:
                break
        
        # update policy
        policy_stable = True    
        for s in S:
            old_action = pi[s]
            actions = A(s)
            pi[s] = actions[
                np.argmax(
                    sum([p(s_,r,s,a) * (r + gamma*V[s]) for s_, r in product(S, R)]) 
                for a in actions
            )]
            if old_action != pi[s]:
                policy_stable = False
        return pi, V



In [21]:
S = set([(l1,l2) for l1 in range(21) for l2 in range(21)])
R = range(0,401, 2)

def poisson(n, lamb=1):
    return ((lamb ** n)/np.math.factorial(n))*np.exp(-lamb) if n >= 0 else 0

"""
Produce possible actions from state s
Here s is an ellement of S
"""
def actions(s):
    return range(-min(5, s[1]), min(5, s[0]) + 1)


"""
We assume that the action for the last state.
last_state: number of cars we have at the two lots
action: number of cars we move from lot 1 to lot 2
reward: amount we make
next state the number of cars we expect to have tomarow
"""
def dymanics(next_state, reward, last_state, action) -> float:
    if (reward - action*2) % 10 != 0:
        # this is not possible since we should have sold a natrual number of cars
        return 0
    num_sold = (reward - action*2) // 10
    # compute the probability
    # sum over disjoint events
    prob = 0
    for sold_at_lot_1 in range(num_sold + 1):
        # if we assume the number of cars sold at one lot is fixed then
        # we can compute the probability.
        # since each one of these events are disjoint we can sum there probabilities.
        sold_at_lot_2 = num_sold - sold_at_lot_1
        num_end_of_day_lot_1 = next_state[0] + action # we assume this includes the newly arived cars
        num_end_of_day_lot_2 = next_state[1] - action
        num_arived_lot_1 = num_end_of_day_lot_1 + sold_at_lot_1 - last_state[0]
        num_arived_lot_2 = num_end_of_day_lot_2 + sold_at_lot_2 - last_state[1]
        prob += (
            poisson(sold_at_lot_1, lamb=3)
            *poisson(sold_at_lot_2, lamb=4)
            *poisson(num_arived_lot_1, lamb=3)
            *poisson(num_arived_lot_2, lamb=2))
    return prob

print(S)

{(7, 3), (16, 9), (19, 4), (17, 20), (20, 7), (18, 19), (8, 5), (9, 0), (10, 7), (0, 17), (14, 1), (12, 17), (15, 4), (13, 20), (3, 2), (4, 5), (16, 0), (19, 13), (17, 13), (20, 14), (18, 10), (8, 12), (9, 9), (10, 14), (8, 18), (11, 15), (9, 19), (14, 8), (12, 8), (15, 13), (13, 13), (2, 18), (0, 14), (3, 11), (1, 15), (4, 12), (2, 12), (5, 1), (3, 17), (16, 7), (6, 14), (19, 18), (17, 6), (7, 15), (18, 5), (10, 9), (11, 4), (9, 20), (14, 19), (12, 7), (10, 19), (15, 10), (13, 6), (0, 5), (1, 0), (4, 11), (2, 7), (5, 10), (6, 1), (4, 17), (7, 4), (5, 20), (16, 20), (19, 1), (17, 17), (8, 0), (15, 19), (11, 19), (12, 20), (16, 11), (6, 18), (19, 6), (17, 10), (20, 1), (18, 17), (8, 7), (9, 6), (10, 5), (11, 8), (0, 19), (14, 7), (12, 19), (1, 18), (15, 6), (13, 18), (0, 9), (3, 4), (4, 7), (5, 6), (16, 2), (19, 15), (17, 3), (7, 16), (20, 8), (18, 8), (8, 14), (9, 15), (10, 12), (8, 20), (11, 1), (9, 17), (14, 14), (12, 10), (15, 15), (13, 11), (2, 16), (0, 0), (3, 13), (1, 13), (4, 14

In [22]:
policy_iteration(S, R, dymanics, actions, 0.9, 0.01)

KeyboardInterrupt: 